![Banner-3.png](<../../images/tasty_bytes/Banner-3.png>)

# Overview
Tasty Bytes is one of the largest food truck networks in the world with localized menu options spread across 30 major cities in 15 countries.<br>**Tasty Bytes is aiming to achieve 25% YoY sales growth over 5 years.**

As Tasty Bytes Data Scientists, we have been asked to support this goal by helping our food truck drivers more intelligently pick where to park for shifts.<br>**We want to direct our trucks to locations that are expected to have the highest sales on a given shift. <br>This will maximize our daily revenue across our fleet of trucks.**

To provide this insight, we will use historical shift sales at each location to build a model. This data has been made available to us in Snowflake.<br> 

Our model will provide the predicted sales at each location for the upcoming shift.

![problem_overview.png](<../../images/tasty_bytes/problem_overview.png>)

**This is an introduction to Snowpark for Snowflake.
We will use Snowpark to:**

- Explore the data
- Perform feature engineering
- Train a model
- Deploy the model in Snowflake


**Why Snowpark?**

- No copies or movement of data
- Maintain governance
- Leverage Snowflake scalable compute
- ...and more!

![snowpark_101.png](<../../images/tasty_bytes/snowpark_101.png>)
Let's get to know Snowpark. We will see that Snowpark makes it easy for Python users to leverage the Snowflake platform. Bringing these users into the Snowflake platform will foster collaboration and  streamline architecture across all users and teams.

## Import Packages
Just like the Python packages we are importing, we will import the Snowpark modules that we need.<br>
**Value**: Snowflake modules provide efficient ways to work with data and functions in Snowflake. 

In [ ]:
# Import Python packages
import pandas as pd
import plotly.express as px
import plotly.io as pio
import json
import sys
import cachetools

# Import Snowflake modules
from snowflake.snowpark import Session
import snowflake.snowpark.functions as F
import snowflake.snowpark.types as T
from snowflake.snowpark import Window

## Connect to Snowflake
Our Snowflake role, **hol_role**, can access the data in the **analytics** schema of the **hol_db** database. We will use the **hol_wh** warehouse that has been created as a dedicated compute for data science workloads.

We will use these parameters and our Snowflake account credentials to connect to Snowflake and create a Snowpark session.<br>
**Value:** Secure and governed. 

In [ ]:
session = Session.builder.getOrCreate()

session.sql("SELECT CURRENT_ROLE()").show()

## Snowpark DataFrame
Let's create a Snowpark DataFrame containing our shift sales data from the **shift_sales_v** view in our Snowflake account using the Snowpark session.table function. A DataFrame is a data structure that contains rows and columns, similar to a SQL table.<br>
**Value:** Familiar representation of data for Python users.

In [ ]:
snowpark_df = session.table("hol_db.analytics.shift_sales_v")

## Preview the Data
With our Snowpark DataFrame defined, let’s use the .show() function to take a look at the first 10 rows. <br>
**Value:** Instant access to data.

In [ ]:
snowpark_df.show()

## Select, Filter, Sort
Notice the Null values for "shift_sales". Let's look at a single location.

To do this, we will make another Snowpark DataFrame, location_df, from the above DataFrame and we will:
1. Select columns
2. Filter to a single location ID
3. Sort by date

**Value**: Efficient transformation pipelines using Python syntax and chained logic.

In [ ]:
# Select
location_df = snowpark_df.select("date", "shift", "shift_sales", "location_id", "city")

# Filter
location_df = location_df.filter(F.col("location_id") == 1135)

# Sort
location_df = location_df.order_by(["date", "shift"], ascending=[0, 0])

# Display
location_df.show(n=20)

We can see that shift sales are populated 8 days prior to the latest date in the data. The **missing values** represent future dates that do not have shift sales yet.


## Snowpark works in two main ways:
1. Snowpark code translated and executed as SQL on Snowflake
2. Python functions deployed in a secure sandbox in Snowflake 

![snowpark_overview.png](<../../images/tasty_bytes/snowpark_overview.png>)

## Explain the Query
Let's look at what was executed in Snowflake to create our location_df DataFrame.

The translated SQL query can be seen in the Snowsight interface under *Activity* in the *Query History* or directly in our notebook by using the explain() function.
![query_history.png](<../../images/tasty_bytes/query_history.png>)

**Value:** Transparent execution and compute usage.

In [ ]:
location_df.explain()

## Compare DataFrame Size
Let's bring a sample of our Snowflake dataset to our Python environment in a pandas DataFrame using the to_pandas() function. We will compare how much memory is used for the pandas DataFrame compared to the Snowpark DataFrame. As we will see, no memory is used for the Snowpark DataFrame in our Python environment. All data in the Snowpark DataFrame remains on Snowflake.<br>

**Value:** No copies or movement of data when working with Snowpark DataFrames.

In [ ]:
# Bring 10,000 rows from Snowflake to pandas
pandas_df = snowpark_df.limit(10000).to_pandas()

# Get Snowpark DataFrame size
snowpark_size = sys.getsizeof(snowpark_df) / (1024*1024)
print(f"Snowpark DataFrame Size (snowpark_df): {snowpark_size:.2f} MB")

# Get pandas DataFrame size
pandas_size = sys.getsizeof(pandas_df) / (1024*1024)
print(f"Pandas DataFrame Size (pandas_df): {pandas_size:.2f} MB")

![data_exploration.png](<../../images/tasty_bytes/data_exploration.png>)
Here, we will use Snowpark to explore our data. A common pattern for exploration is to use Snowpark to manipulate our data and then bring an aggregate table to our Python environment for visualization. <br>

**Value:** 
   - Native Snowflake performance and scale for aggregating large datasets. 
   - Easy transfer of aggregate data to the client-side environment for visualization.
   
<br>As we explore our data, we will highlight what is being done in Snowflake and what we are transferring to our client-side environment (Python notebook environment) for visualization.

## How many rows are in our data?
This will give us an idea of how we might need to approach working with this data. Do we have enough data to build a meaningful model? What compute might be required? Will we need to sample the data?<br>
**What's happening where?:** Rows counted in Snowflake. No data transfer.

In [ ]:
snowpark_df.count()

## Let's calculate some descriptive statistics.
We use the Snowpark describe() function to calculate summary statistics and then bring the aggregate results into a pandas DataFrame to visualize in a formatted table.<br>
**What's happening where?:** Summary statistics calculated in Snowflake. Transfer aggregate summary statistics for client-side visualization.

In [ ]:
snowpark_df.describe().to_pandas()

## What are the numeric columns?
We want to understand the data types in our data and how we might need to handle them in preparation for modeling. For numeric columns, this could include normalizing the data to the same scale or applying a transformation to change the distribution.<br>
**What's happening where?:** The Snowflake table schema is used to get metadata information about the data. No data transfer.

In [ ]:
# Define Snowflake numeric types
numeric_types = [T.DecimalType, T.DoubleType, T.FloatType, T.IntegerType, T.LongType]

# Get numeric columns
numeric_columns = [col.name for col in snowpark_df.schema.fields if type(col.datatype) in numeric_types]
numeric_columns

## What are the categorical columns?
Our model requires all features to be numeric. We want to identify columns that we will need to transform to a numeric representation if we would like to use them as features in our model.<br>
**What's happening where?:** The Snowflake table schema is used to get metadata information about the data. No data transfer.

In [ ]:
# Define Snowflake categorical types
categorical_types = [T.StringType]

# Get categorical columns
categorical_columns = [col.name for col in snowpark_df.schema.fields if type(col.datatype) in categorical_types]
categorical_columns

## What are the average shift sales (USD) by city?
Here, we are trying to understand what a "normal" shift sale looks like. What is the span of averages across cities? Are there any outlier cities that should be removed from our training data? Is there anything unexpected in the order of cities sorted by their average shift sales?<br>
**What's happening where?:** Average sales by city calculated in Snowflake. Transfer city averages for client-side visualization.

In [ ]:
# Group by city and average shift sales
analysis_df = snowpark_df.group_by("city").agg(F.mean("shift_sales").alias("avg_shift_sales"))

# Sort by average shift sales
analysis_df = analysis_df.sort("avg_shift_sales", ascending=True)

# Pull to pandas and plot
analysis_df.to_pandas().plot.barh(x="CITY", y="AVG_SHIFT_SALES")

## Looking at Vancouver, how many locations are there?
We are going to use Vancouver, Canada as our test city when predicting the best location for future shifts. Let's get to know locations and shift sales in that city. First, we will see how many location options there are in Vancouver for a food truck to park.<br>
**What's happening where?:** Data filtered, averages calculated by location, and locations counted in Snowflake. No data transfer.

In [ ]:
# Filter to Vancouver
analysis_df = snowpark_df.filter(F.col("city") == "Vancouver")

# Group by location and average shift sales
analysis_df = analysis_df.group_by("location_id").agg(F.mean("shift_sales").alias("avg_shift_sales"))

# Get the location count
print("Vancouver location count:", analysis_df.count())

## What is the distribution of average shift sales (USD) by location in Vancouver?
The x-axis represents average shift sales (USD) and the y-axis represents the count of locations with that average. The highest bars indicate the most common average shift sales. We will keep this in mind when evaluating predictions.<br>
**What's happening where?:** Data filtered and averages calculated by locations in Snowflake. Transfer location averages for client-side visualization.<br>
**Value:** Aggregate where the data is stored using native Snowflake performance and scale. Easy transfer of aggregate data to client-side applications.

In [ ]:
ax = analysis_df.to_pandas().hist(column="AVG_SHIFT_SALES", bins=20)

![feature_engineering.png](<../../images/tasty_bytes/feature_engineering.png>)
We will create a feature engineering pipeline by chaining transformations to prepare our data for model training.<br>
**Value:** The Snowpark syntax makes pipelines easy to implement and understand. The syntax also allows for easy migration of Spark pipelines to Snowflake.<br>
**All transformations for feature engineering in this notebook will be executed on Snowflake compute.**

Notice what we haven't had to do? No tuning, maintenance, or operational overhead. We just need a role, warehouse, and access to the data.<br>
**Value**: Near-zero maintenance. Focus on the work that brings value.

## Create a Rolling Average Feature
We will use a Snowflake window function to get a **rolling shift average by location** over time. Window functions allow us to aggregate on a "moving" group of rows.

**Step 1. Create a Window**

Our window will partition the data by location and shift. It will order rows by date. It will include all rows prior to the current date of the observation it is aggregating for.

In [ ]:
window_by_location_all_days = (
    Window.partition_by("location_id", "shift")
    .order_by("date")
    .rows_between(Window.UNBOUNDED_PRECEDING, Window.CURRENT_ROW - 1)
)

**Step 2. Aggregate across the Window**

In [ ]:
snowpark_df = snowpark_df.with_column(
    "avg_location_shift_sales", 
    F.avg("shift_sales").over(window_by_location_all_days)
)

## Impute Missing Values
The rolling average feature we just created is missing if there are no prior shift sales at that location. We will replace those missing values with 0.

In [ ]:
snowpark_df = snowpark_df.fillna(value=0, subset=["avg_location_shift_sales"])

## Encode Categorical Columns
Categorical columns need to be represented as numeric in our model. We will use binary encoding for the **shift** column by replacing:
- "AM" with 1
- "PM" with 0

In [ ]:
snowpark_df = snowpark_df.with_column("shift", F.iff(F.col("shift") == "AM", 1, 0))

## Filter to Historical Data
Our data includes placeholders for future data with missing shift sales. The **future data** represents the next 7 days of shifts for all locations. The **historical data** has shift sales for all locations where a food truck parked during a shift. We will only use historical data when training our model and will filter out the dates where the **shift_sales** column is missing.

In [ ]:
historical_snowpark_df = snowpark_df.filter(F.col("shift_sales").is_not_null())

## Drop Columns
Drop ID columns that will not be used in the model.

In [ ]:
historical_snowpark_df = historical_snowpark_df.drop("location_id", "city", "date")

## Split Data into Training and Testing
We will use 80% of the data for model training and 20% for testing.<br>

In [ ]:
session.sql("USE SCHEMA hol_db.analytics").collect()
train_snowpark_df, test_snowpark_df = historical_snowpark_df.randomSplit([0.8, 0.2])

## Save Tables in Snowflake
We will save our training and test datasets to the **analytics schema** in our Snowflake account.<br>
**Value:** Eliminate redundant data processing. These tables can be re-used to train more models beyond what we are training today.

In [ ]:
# Save training data
train_snowpark_df.write.mode("overwrite").save_as_table("hol_db.analytics.shift_sales_train")

# Save test data
test_snowpark_df.write.mode("overwrite").save_as_table("hol_db.analytics.shift_sales_test")

![model_training.png](<../../images/tasty_bytes/model_training.png>)
We will now use our training data to train a linear regression model on Snowflake. 

Recall from above, the two main ways that Snowpark works: 
1. Snowpark code translated and executed as SQL on Snowflake
2. Python functions deployed in a secure sandbox in Snowflake 

We will be leveraging the deployment of Python functions into Snowflake for training and model deployment.

![end_to_end_ml.png](<../../images/tasty_bytes/end_to_end_ml.png>)
Here, we see a typical data science workflow. We are finished **preparing** our data and now move on to **training in a Python stored procedure on Snowflake**. The model created from this stored procedure will be our tool for automating decisions around truck locations to maximize our revenue. We'll surface the predicted sales (model inference) on future data using a **Python user-defined function** to drive the decisions.

**Snowflake Stored Procedures** work well for training because they can read data, hold an entire table in memory to find patterns, and write files (e.g. model files) back to the Snowflake database.

**Snowflake User-Defined Functions** work well for inference because they return a single value for each row passed to the user-defined function. Because of this, they can easily be distributed to provide fast results.

**Value**: Effortless, scalable, and secure processing **without data movement** across compute environments.

## Create a Stage
We will use this stage to host user-defined functions, stored procedures, and model files on Snowflake.

In [ ]:
# Create stage
session.sql("CREATE STAGE IF NOT EXISTS hol_db.analytics.model_stage").collect()

## Create a Stored Procedure for Model Training

**Step 1. Create a Function for Training a Model**<br>
This function trains a Scikit-learn linear regression model and saves the model to a stage. Linear regression finds a line that best fits the data points used in training. We then use that line as an estimation of where output values will be for future scenarios.

Here, training will use historical shift sales and features in our data to predict future shift sales of locations where our food trucks can park.
- **Inputs:** Training table name on Snowflake, feature column names, target column names, file name for the saved model
- **Outputs:**  Feature weights of the trained model *(or feature contributions to the predicted value)*

In [ ]:
def train_linreg(
    session: Session,
    training_table: str,
    feature_cols: list,
    target_col: str,
    model_name: str,
) -> T.Variant:

    # Import packages
    from sklearn.linear_model import LinearRegression
    from joblib import dump

    # Get training data
    df = session.table(training_table).to_pandas()

    # Set inputs X and outputs y
    X = df[feature_cols]
    y = df[target_col]

    # Train model
    model = LinearRegression().fit(X, y)

    # Get feature weights
    feature_weights = pd.DataFrame({"Feature": model.feature_names_in_, "Weight": model.coef_}).to_dict()

    # Save model
    dump(model, "/tmp/" + model_name)
    session.file.put(
        "/tmp/" + model_name,
        "@HOL_DB.ANALYTICS.MODEL_STAGE",
        auto_compress=False,
        overwrite=True
    )

    # Return feature contributions
    return feature_weights


**Step 2. Register the Function on Snowflake**<br>
To register the function on Snowflake as a stored procedure, specify what Python packages are required in the function. Here we specify:
- Snowpark
- Scikit-learn (for training our model)
- Joblib (for creating a model file)

Scikit-learn is a popular Python library for Machine Learning. We will be able to leverage its functionality in Snowflake in our deployed stored procedure.

**Value:** Easy access to hundreds of Python packages with automated dependency management.<br>
*Python packages available in Snowflake:* https://repo.anaconda.com/pkgs/snowflake/

In [ ]:
train_linreg_snowflake = session.sproc.register(
    func=train_linreg,
    name="sproc_train_linreg",
    is_permanent=True,
    replace=True,
    stage_location="@HOL_DB.ANALYTICS.MODEL_STAGE",
    packages=["snowflake-snowpark-python", "scikit-learn", "joblib"]
)


## Model Training
We will now train our model on Snowflake using our stored procedure. If more memory was required, a Snowpark Optimized Warehouse could be used. In this case, a standard Snowflake warehouse is sufficient.

*Documentation for Snowpark Optimized Warehouses:* https://docs.snowflake.com/en/developer-guide/snowpark/python/python-snowpark-training-ml.html<br>
**Value:** Always access the compute you need - no bottlenecks!

**Call the Training Stored Procedure**<br><br>
We will call our stored procedure and specify our training table location, feature and target columns, and our saved model name. This will perform model training in Snowflake.<br>
**Value:** Secure - Python stored procedures and user-defined functions execute in highly secure sandboxed environments.

In [ ]:
# Specify inputs
training_table = "hol_db.analytics.shift_sales_train"
model_name = "linreg_location_sales_model.sav"
feature_cols = [
    "MONTH",
    "DAY_OF_WEEK",
    "LATITUDE",
    "LONGITUDE",
    "CITY_POPULATION",
    "AVG_LOCATION_SHIFT_SALES",
    "SHIFT",
]
target_col = "SHIFT_SALES"


# Call the training stored procedure
feature_contributions = train_linreg_snowflake(
    session, training_table, feature_cols, target_col, model_name
)

## Examine Feature Contributions
Our stored procedure returns the feature contributions. Here we can see which features have the largest impact on shift sales predictions. The prediction provided by the linear regression model is a summation of the feature values multiplied by their respective weights (plus an additional bias term). We can see that people are hungrier in the afternoon!<br>
**Value**: Training in a stored procedure isn't a black-box process. We can return the insight we need (in this case, the feature weights) or write to a table in Snowflake.

In [ ]:
pd.DataFrame(eval(feature_contributions)).sort_values(by="Weight")

## View the Saved Model
Let's make sure our model is saved to the stage on Snowflake. We will list the files on the stage and see when they were last modified.

In [ ]:
pd.DataFrame(session.sql("LIST @HOL_DB.ANALYTICS.MODEL_STAGE").collect())

![model_deployment.png](<../../images/tasty_bytes/model_deployment.png>)
We need to make our model available in a central location where it can be used by other users and teams. 
- The application development team will need to integrate it with the truck driver app to surface recommendations. 
- The analyst team will use it to benchmark and measure the impact on sales. 
- As data scientists, we will want to monitor the performance and work to improve the model. 

What better location to deploy the model, than with the data being used by these teams?<br>
**Value:** Democratize access to models and predictions.

## Create a User-Defined Function for Model Inference
**Step 1. Create a Function for Model Inference**<br>
This function loads the saved model and predicts shift sales from input features
- **Inputs:** Features
- **Outputs:**  Predicted shift sales

Because we specified the input as a pandas DataFrame and output as a pandas Series, the user-defined function will **execute in batches** instead of one row at a time. In many cases, this reduces the overall execution time for the query calling the user-defined function.

We also use the Python library cachetools to cache the model that is returned from the load_model function. This ensures that the **model is loaded once** per user-defined function call, instead of for each row or batch in the data.

**Value:** Fast and efficient predictions.

In [ ]:
# Function to load the model from file and cache the result
@cachetools.cached(cache={})
def load_model(filename):
    
    # Import packages
    import sys
    import os
    import joblib
    
    # Get the import directory where the model file is stored
    import_dir = sys._xoptions.get("snowflake_import_directory")
    
    # Get the import directory where the model file is stored
    if import_dir:
        with open(os.path.join(import_dir, filename), 'rb') as file:
            m = joblib.load(file)
            return m

# Function to predict shift sales
def linreg_predict_location_sales(X: pd.DataFrame) -> pd.Series:
    
    # Load the model
    model = load_model("linreg_location_sales_model.sav")

    # Get predictions
    predictions = model.predict(X)

    # Return rounded predictions
    return predictions.round(2)

**Step 2. Register the Function on Snowflake**<br>
To register the function on Snowflake as a user-defined function, we specify the Python packages required. Here we specify:
- Scikit-learn (for making predictions)
- Joblib (for loading the model from file)
- Cachetools (for caching the loaded model)

Additionally, we specify which files to import with the user-defined function. Here we specify our trained model file as an import.<br>
**Value:** Simple and painless model deployment.

In [ ]:
session.udf.register(
    func=linreg_predict_location_sales,
    name="udf_linreg_predict_location_sales",
    stage_location="@HOL_DB.ANALYTICS.MODEL_STAGE",
    input_types=[T.FloatType()] * len(feature_cols),
    return_type=T.FloatType(),
    replace=True,
    is_permanent=True,
    imports=["@HOL_DB.ANALYTICS.MODEL_STAGE/linreg_location_sales_model.sav"],
    packages=["scikit-learn", "joblib", "cachetools"]
)

## Predict!
Now that our inference user-defined function is deployed, we can use it in Snowflake on our data to get shift sales predictions.

**Scale Up Snowflake Compute**<br>
We can perform distributed model inferencing across the nodes of our Snowflake warehouse by scaling up to a multi-node warehouse. By scaling up to a medium warehouse, the processing will distribute across all four warehouse nodes instead of using single-threaded execution on an x-small warehouse with 1 node. 
We haven't required a medium warehouse up to this point but can easily adjust our warehouse now to meet the compute requirements for this portion of our workflow.<br>

**Value:** Cost-effective performance by adapting processing to machine learning needs.

In [ ]:
session.sql("ALTER WAREHOUSE HOL_WH SET WAREHOUSE_SIZE = MEDIUM").collect()

**Call the Inference Function**

In [ ]:
train_pred = train_snowpark_df.select(
    "shift_sales",
    F.call_udf("udf_linreg_predict_location_sales", [F.col(c) for c in feature_cols]).alias("prediction")
)
train_pred.show()

## Evaluate
We will evaluate our model using Root-Mean-Square Error (RMSE) for both our training and test data. We will get predictions with our user-defined function and compare them to the actual historical shift sales.

**Value:** Calculation of our evaluation metric and model inference happens on Snowflake compute. No data moves outside of Snowflake and governance is maintained.

In [ ]:
# Train evalutation RMSE
rmse_train = train_pred.select(F.sqrt(F.mean((F.col("shift_sales") 
                                              - F.col("prediction"))**2)))

print("Training data RMSE:", round(rmse_train.collect()[0][0]))

# Test evaluation RMSE
rmse_test = test_snowpark_df.select(F.sqrt(F.mean((F.col("shift_sales") 
                                                   - F.call_udf("udf_linreg_predict_location_sales",
                                                                [F.col(c) for c in feature_cols]))**2)))

print("Test data RMSE:", round(rmse_test.collect()[0][0]))

![model_utilization.png](<../../images/tasty_bytes/model_utilization.png>)
Now that our model is built and deployed, let's see it in action! We will find the best place to park in Vancouver for tomorrow morning's shift.

![problem_overview.png](<../../images/tasty_bytes/problem_overview.png>)
## Predict Location Sales for the Next Shift
We will filter to the morning shift of the first future date in our data and Vancouver locations. We then call our inference user-defined function to get predicted shift sales at each location.

In [ ]:
# Get the date to predict
date_tomorrow = snowpark_df.filter(F.col("shift_sales").is_null()).select(F.min("date")).collect()[0][0]
# Filter to tomorrow's date and the morning shift in Vancouver
location_predictions_df = snowpark_df.filter((F.col("date") == date_tomorrow) 
                                             & (F.col("shift") == 1) 
                                             & (F.col("city")=="Vancouver"))

# Get predictions
location_predictions_df = location_predictions_df.select(
    "location_id", 
    "latitude", 
    "longitude",
    F.call_udf("udf_linreg_predict_location_sales", [F.col(c) for c in feature_cols]).alias("predicted_shift_sales")
)

## Visualize on a Map
The yellow dots indicate higher predicted sales locations and the purple dots indicate lower predicted sales. We will use this insight to ensure that our drivers are parking at the high-value locations. <br>
**Value:** Updated predictions readily available to drive towards our corporate goals.

In [ ]:
# Pull location predictions into a pandas DataFrame
predictions_df = location_predictions_df.to_pandas()

# Visualize on a map
pio.renderers.default='notebook'

fig = px.scatter_mapbox(
    predictions_df, 
    lat="LATITUDE", 
    lon="LONGITUDE", 
    hover_name="LOCATION_ID", 
    size="PREDICTED_SHIFT_SALES",
    color="PREDICTED_SHIFT_SALES",
    zoom=8, 
    height=800,
    width=1000
)

fig.update_layout(mapbox_style="open-street-map")
fig.show()

![blue_frostbyte_primary_logo.png](<../../images/tasty_bytes/blue_frostbyte_primary_logo.png>)
**Quickstart: 		Tasty Bytes - Snowpark 101 for Data Science**<br />
Script: 	tasty_bytes_snowpark_101.ipynb <br />   	
Create Date:    2023-05-19 <br />
Author:         Marie Coolsaet <br />

**Description:**<br />
A linear regression model is trained and deployed in Snowflake to predict the best location for food trucks to park during their shifts.


**Summary of Changes:**<br />

| Date(yyyy-mm-dd) | Author         | <div style="width:290px">Comments</div>                        |
| :---             | :---           | :---                                                  |
|2023-05-19         |Marie Coolsaet | Initial Quickstart Release|